In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

spark = SparkSession.builder \
  .remote("sc://192.168.1.7:15002") \
  .appName("UDFTransformation") \
  .config("spark.sql.ansi.enabled", "false") \
  .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
  .getOrCreate()

# limit() shows a nice HTML table in Jupyter, while show() prints plain text  
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

spark

In [2]:
DataStreamReader = spark.readStream \
  .format("socket") \
  .option("host", "192.168.1.7") \
  .option("port", 9999)

linesDF = DataStreamReader.load()
lines_writer = linesDF.writeStream \
  .outputMode("append") \
  .format("memory") \
  .queryName("lines_table")

DataStreamReader

In [3]:
def stop_queries():
  for query in spark.streams.active:
    query.stop()

# Task 1

In [20]:
@udf
def name_transform(sentence, first="André", last="Plancha"):
  words = sentence.split()
  return ','.join([f"{first}{word}{last}" for word in words])

transformedDF = linesDF.withColumn("value", name_transform(linesDF.value))


stop_queries()

task1_writer = transformedDF.writeStream.outputMode("append").format("memory").queryName("task1_table")
task1_query = task1_writer.start()

task1_writer

```console
ssh plancha@192.168.1.7 -t /usr/bin/nc -lk 9999
plancha@192.168.1.7's password: 
Sweden is a good country
Finland is a better country
Denmark is an awesome country
I love programming in Spark
Data is the new oil
```

In [21]:
task1_query.stop()

spark.sql("SELECT * FROM task1_table").show(20, truncate=False)

+-----------------------------------------------------------------------------------------+
|value                                                                                    |
+-----------------------------------------------------------------------------------------+
|AndréSwedenPlancha,AndréisPlancha,AndréaPlancha,AndrégoodPlancha,AndrécountryPlancha     |
|AndréFinlandPlancha,AndréisPlancha,AndréaPlancha,AndrébetterPlancha,AndrécountryPlancha  |
|AndréDenmarkPlancha,AndréisPlancha,AndréanPlancha,AndréawesomePlancha,AndrécountryPlancha|
|AndréIPlancha,AndrélovePlancha,AndréprogrammingPlancha,AndréinPlancha,AndréSparkPlancha  |
|AndréDataPlancha,AndréisPlancha,AndréthePlancha,AndrénewPlancha,AndréoilPlancha          |
+-----------------------------------------------------------------------------------------+

